In [2]:
from PIL import Image
import requests
import torch
#from torchvision import io
from typing import Dict
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [3]:
prompt = "Process this image in two steps: Step 1: Turn the text in this picture into markdown. Indicate italics. \
            Indent lines which discuss folder contents as unordered lists Step 2: Convert the markdown text that you created in step 1 into JSON. \
            Use the heading texts as keys, and the folder details as nested numbered lists\
            Your output should consist of the markdown text, then a fenced JSON code block"


In [16]:
%%time

#Load the model in half-precision on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

# Image
url = "./sample-images/pg132_a.jpeg"
image = Image.open(url)

conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {"type": "text", "text": prompt},
        ],
    }
]


# Preprocess the inputs
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
# Excepted output: '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>\n<|im_start|>assistant\n'

inputs = processor(
    text=[text_prompt], images=[image], padding=True, return_tensors="pt"
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
output_ids = model.generate(**inputs, max_new_tokens=1024) 
## Increasing the number of new tokens, increases the number of words recognised by the model with trade-off of speed
## 1024 new tokens was capable of reading upto 70% of the input image (pg132_a.jpeg)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]
output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)
print(output_text)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:15<00:00, 15.06s/it]


['### Dicotyledones\n\n#### ACERACEAE\n\n**Acer campestre L.**\n\n- 1 folder. Acer campestre [TA]\n\n**Acer pseudoplatanus L.**\n\n- 2 folders.\n\n  - Folder 1. Acer Pseudo-Platanus [G]. i. "Maple. Bulls: [Bulstrode] Park" [JL]\n  - Folder 2. Acer Pseudo-Platanus [TA].\n\n#### AMARANTHACEAE\n\n**Amaranthus lividus L., Flora Europaea 1: 110 (1964)**\n\n- 1 folder. Amaranthus Blitum [TA].\n\n  - i. Cites Ray\'s Syn. 1957. ii. "Blite Amaranth. Aug! It is often found on Dunghills in the neighbourhood of London. I gather\'d this on a Dunghill at Fulham near London" [JL]. iii. "Amaranthus Blitum. Monoec: 5. and:" [JL]\n\n#### ARALIACEAE\n\n**Hedera helix L.**\n\n- 1 folder. Hedera Helix [TA].\n\n#### ARISTOLOCHIACEAE\n\n**Asarum europaeum L.**\n\n- 1 folder. Asarum europaeum [TA].\n\n  - i. "Asarum europaeum Lin: Asarabacca. Anglis. ex horto" [JL].\n\n#### BALSAMINACEAE\n\n**Impatiens noli-tangere L.**\n\n- 1 folder. Impatiens Noli me tangere [G].\n\n#### BETULACEAE\n\n**Alnus glutinosa (L.)

In [17]:
generated_ids

[tensor([14374,   422, 86437,  ...,    77,   355, 34069], device='cuda:0')]